This notebook goes through the following with TensorFlow:

- build a transfer learning feature extraction model using TensorFlow Hub
- use TensorBoard callback to track and compare model training results

In [ ]:
# Add timestamp
import datetime
print(f"Notebook last run (end-to-end): {datetime.datetime.now()}")

Notebook last run (end-to-end): 2024-01-04 02:29:49.917724


In [ ]:
# Check if we're using GPU
!nvidia-smi

Thu Jan  4 02:29:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## download data (10% of the food vision data)

In [ ]:
import zipfile

# Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
# Unzip data
zip_data = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_data.extractall()
zip_data.close()

--2024-01-04 02:32:25--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.96.207, 108.177.119.207, 108.177.127.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.96.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  42.7MB/s    in 4.4s    

2024-01-04 02:32:30 (36.7 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



## Check data

In [ ]:
import os

for root, dirs, files in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirs)} directories and {len(files)} images under '{root}'.")

There are 2 directories and 0 images under '10_food_classes_10_percent'.
There are 10 directories and 0 images under '10_food_classes_10_percent/train'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images under 

## Normalization and create data batches

In [ ]:
# Normalization and create data batches
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"
batch_size = 32
img_size = (224, 224)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=img_size,
                                               batch_size=batch_size,
                                               class_mode="categorical")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=img_size,
                                             batch_size=batch_size,
                                             class_mode="categorical")

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


## Set up TensorBoard callbacks

[Callbacks](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks) are extra functionality you can add to your models to be performed during or after training. Some of the most popular callbacks include:

- Experiment tracking with TensorBoard - **log the performance** of multiple models and then view and compare these models in a visual way on TensorBoard. Helpful to compare the results of different models on your data.
- Model checkpointing - save your model as it trains so you can stop training if needed and come back to continue off where you left. Helpful if training takes a long time and can't be done in one sitting.
- Early stopping - leave your model training for an arbitrary amount of time and have it stop training automatically when it ceases to improve. Helpful when you've got a large dataset and don't know how long training will take.




### Create a TensorBoard callback

We want to save a model's performance logs to a directory named [dir_name]/[experiment_name]/[current_timestamp], where:

- dir_name is the overall logs directory
- experiment_name is the particular experiment
- current_timestamp is the time the experiment started based on Python's datetime.datetime().now()

In [11]:
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.dateime().now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  return tensorboard_callback

## Creating models using TensorFlow Hub